In [1]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

import tensorflow as tf
#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from tensorflow.keras.layers import concatenate
from deepforest import CascadeForestClassifier
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [17]:

input = Input(shape = (224,224,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)

# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
x = mobilnet_block(x, filters = 128, strides = 1)
x = mobilnet_block(x, filters = 256, strides = 2)
x = mobilnet_block(x, filters = 256, strides = 1)
x = mobilnet_block(x, filters = 512, strides = 2)
for _ in range (5):
    x = mobilnet_block(x, filters = 512, strides = 1)
x = mobilnet_block(x, filters = 1024, strides = 2)
x = mobilnet_block(x, filters = 1024, strides = 1)
x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
x = tf.keras.layers.Dropout(0.4)(x)
output= Flatten()(x)
output = Dense (256, activation = "relu", kernel_regularizer='l2')(output)
output = Dense (1, activation = "sigmoid")(output)
model = Model(inputs=input, outputs=output)
# model.fit()


In [6]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/Gas Sensors Data - Sheet1 (3) (1).csv")
data_dir = '/Users/Eddie/Downloads/No Gas 4/No Gas'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        #print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/No Gas 4/No Gas/'+image)
        img.resize(224,224,3)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)

176


In [7]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

# print(testy)
newtesty = []
for value in testy:
    value = float(value)
    newtesty.append(value)
# print(newtesty)
newtrainy = []
for value in trainy:
    value = float(value)
    newtrainy.append(value)


(140, 2)
(36, 2)
(140,)
(36,)


In [18]:
print(newtrainy)
#model.load('/Users/Eddie/Downloads/alexnet')
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001), loss='binary_crossentropy',metrics = ['acc'])

[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]


In [19]:
hist = model.fit(trainImagesX,trainy,validation_data=(testImagesX,testy),epochs=300,batch_size = 20)

Epoch 1/300
7/7 [==============================] - 11s 613ms/step - loss: 5.6779 - acc: 0.4500 - val_loss: 5.6310 - val_acc: 0.6944
Epoch 2/300
7/7 [==============================] - 4s 531ms/step - loss: 5.5766 - acc: 0.6071 - val_loss: 5.6206 - val_acc: 0.6944
Epoch 3/300
7/7 [==============================] - 4s 527ms/step - loss: 5.5768 - acc: 0.6143 - val_loss: 5.6066 - val_acc: 0.6944
Epoch 4/300
7/7 [==============================] - 4s 525ms/step - loss: 5.5224 - acc: 0.6929 - val_loss: 5.5927 - val_acc: 0.6944
Epoch 5/300
7/7 [==============================] - 4s 522ms/step - loss: 5.4477 - acc: 0.7786 - val_loss: 5.5797 - val_acc: 0.6944
Epoch 6/300
7/7 [==============================] - 4s 524ms/step - loss: 5.4401 - acc: 0.7786 - val_loss: 5.5626 - val_acc: 0.6944
Epoch 7/300
7/7 [==============================] - 4s 525ms/step - loss: 5.4077 - acc: 0.7857 - val_loss: 5.5469 - val_acc: 0.6944
Epoch 8/300
7/7 [==============================] - 4s 523ms/step - loss: 5.3643 - 

KeyboardInterrupt: 

In [21]:

performance = model.predict(testImagesX)
performance.round()
actual = []
for value in performance: 
    #print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)

print(actual)
acutal = np.array(actual)
actual.append(1)
testy = np.append(testy,0)
accuracy = accuracy_score(testy,actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

2/2 [==============================] - 0s 33ms/step
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Accuracy: 0.810811
Precision: 0.833333
Recall: 0.454545
F1 score: 0.588235
